In [ ]:
from haversine import haversine
import xmltodict
from sklearn.neighbors import KDTree
import time
import numpy as np

: 

In [ ]:
s = time.time()
doc = {}
with open('data/map.graphml') as fd:
    doc = xmltodict.parse(fd.read())
print(time.time()-s)

: 

In [3]:
def getLatLon(OSMId):
    lat, lon = 0, 0
    nodes = doc['graphml']['graph']['node']
    for eachNode in range(len(nodes)):
        if (nodes[eachNode]["@id"] == str(OSMId)):
            lat = float(nodes[eachNode]["data"][0]["#text"])
            lon = float(nodes[eachNode]["data"][1]["#text"])
    return (lat, lon)

In [4]:
res=getLatLon("1947936514")

In [5]:
res

(40.5207299, -74.4575536)

In [6]:
def getOSMId(lat, lon):
    OSMId = 0
    nodes = doc['graphml']['graph']['node']
    for eachNode in range(len(nodes)):
        if (nodes[eachNode]["data"][0]["#text"] == str(lat)):
            if (nodes[eachNode]["data"][1]["#text"] == str(lon)):
                OSMId = nodes[eachNode]["data"][2]["#text"]

    return OSMId

In [13]:
res=getOSMId("40.49840455168209","-74.44476100847496")

In [14]:
res

0

In [15]:
def calculateHeuristic(curr, destination):
    return (haversine(curr, destination))

In [16]:
res=calculateHeuristic([27.1767,78.0081],[30.9010,75.8573])

In [17]:
res

463.88359865356284

In [18]:
def getNeighbours(OSMId, destinationLetLon):
    neighbourDict = {}
    tempList = []
    edges = doc['graphml']['graph']['edge']
    for eachEdge in range(len(edges)):
        if (edges[eachEdge]["@source"] == str(OSMId)):
            temp_nbr = {}

            neighbourCost = 0
            neighbourId = edges[eachEdge]["@target"]
            neighbourLatLon = getLatLon(neighbourId)

            dataPoints = edges[eachEdge]["data"]
            for eachData in range(len(dataPoints)):
                if (dataPoints[eachData]["@key"] == "d12"):
                    neighbourCost = dataPoints[eachData]["#text"]

            neighborHeuristic = calculateHeuristic(
                neighbourLatLon, destinationLetLon)

            temp_nbr[neighbourId] = [neighbourLatLon,
                                     neighbourCost, neighborHeuristic]
            tempList.append(temp_nbr)

    neighbourDict[OSMId] = tempList
    return (neighbourDict)


def getNeighbourInfo(neighbourDict):
    neighbourId = 0
    neighbourHeuristic = 0
    neighbourCost = 0
    for key, value in neighbourDict.items():

        neighbourId = key
        neighbourHeuristic = float(value[2])
        neighbourCost = float(value[1])/1000
        neighbourLatLon = value[0]

    return neighbourId, neighbourHeuristic, neighbourCost, neighbourLatLon


In [19]:
res=getNeighbours('1947936514',[30.9010,75.8573])

In [20]:
res

{'1947936514': [{'1947936312': [(40.5211255, -74.4587767),
    '112.354',
    11505.752210152154]},
  {'1947936754': [(40.5216943, -74.4570038), '116.873', 11505.629838671599]}]}

In [15]:
res=getNeighbourInfo({'1947936514': [{'1947936312': [(40.5211255, -74.4587767),
    '112.354',
    11505.752210152154]},
  {'1947936754': [(40.5216943, -74.4570038), '116.873', 11505.629838671599]}]})

IndexError: list index out of range

In [22]:
def getKNN(pointLocation):
    nodes = doc["graphml"]["graph"]["node"]
    locations = []
    for eachNode in range(len(nodes)):
        locations.append(
            (nodes[eachNode]["data"][0]["#text"], nodes[eachNode]["data"][1]["#text"]))

    locations_arr = np.asarray(locations, dtype=np.float32)
    point = np.asarray(pointLocation, dtype=np.float32)

    tree = KDTree(locations_arr, leaf_size=2)
    dist, ind = tree.query(point.reshape(1, -1), k=3)

    nearestNeighbourLoc = (
        float(locations[ind[0][0]][0]), float(locations[ind[0][0]][1]))

    return nearestNeighbourLoc

In [24]:
res=getKNN([40.49840455168209,-74.44476100847496])

In [25]:
res

(40.4983425, -74.444755)

In [26]:
def getResponsePathDict(paths, source, destination):
    finalPath = []
    child = destination
    parent = ()
    cost = 0
    while (parent != source):
        tempDict = {}
        cost = cost + float(paths[str(child)]["cost"])
        parent = paths[str(child)]["parent"]
        parent = tuple(float(x) for x in parent.strip('()').split(','))

        tempDict["lat"] = parent[0]
        tempDict["lng"] = parent[1]

        finalPath.append(tempDict)
        child = parent

    return finalPath, cost